In [2]:
import gc
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from scipy.stats import skew, boxcox
from sklearn.preprocessing import StandardScaler

C:\Users\Jenny\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
shift =200

In [4]:
train = pd.read_csv('C:\\hudsondata\\Machine Learning\\Kaggle\AllState\\train.csv')
test = pd.read_csv('C:\\hudsondata\\Machine Learning\\Kaggle\AllState\\test.csv')

In [5]:
numeric_feats = [x for x in train.columns[1:-1] if 'cont' in x]

In [6]:
cats = [x for x in train.columns[1:-1] if 'cat' in x]

In [7]:
ntrain = train.shape[0]
test['loss']=0
train_test = pd.concat((train,test)).reset_index(drop=True)

In [8]:
skewed_feats = train_test[numeric_feats].apply(lambda x: skew(x.dropna()))

In [9]:
skewed_feats = skewed_feats[abs(skewed_feats)>0.25]

In [10]:
skewed_feats= pd.DataFrame(skewed_feats).reset_index()['index']

In [11]:
for feats in skewed_feats:
    train_test[feats] = train_test[feats] + 1

In [12]:
for feats in skewed_feats:
    train_test[feats], lam = boxcox(train_test[feats])

In [14]:
for col in cats:
    train_test[col] = pd.factorize(train_test[col], sort=True)[0]

In [16]:
train_test.loss = np.log(train_test.loss + shift)

In [17]:
ss = StandardScaler()

In [18]:
train_test[numeric_feats] =ss.fit_transform(train_test[numeric_feats].values)

In [20]:
train = train_test.iloc[:ntrain, :].copy()
test = train_test.iloc[ntrain:, :].copy()
test.drop('loss', inplace=True, axis=1)

In [21]:
print('Median Loss:', train.loss.median())

('Median Loss:', 7.747411156506141)


In [22]:
print('Mean Loss:', train.loss.mean())

('Mean Loss:', 7.799837008871419)


In [23]:
xgb_params = {
        'seed': 0,
        'colsample_bytree': 0.7,
        'silent': 1,
        'subsample': 0.7,
        'learning_rate': 0.1,
        'objective': 'reg:linear',
        'max_depth': 12,
        'min_child_weight': 100,
        'booster': 'gbtree',
    }

In [24]:
best_nrounds = 350

In [25]:
allpredictions = pd.DataFrame()

In [27]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x = preds-labels
    grad = con*x / (np.abs(x)+con)
    hess = con**2 / (np.abs(x)+con)**2
    return grad, hess

In [29]:
def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-shift,
                                      np.exp(yhat)-shift)

In [37]:
from sklearn.cross_validation import train_test_split

In [38]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,0,1,0,1,0,0,0,0,1,...,1.110234,-0.871854,-0.989339,1.123537,1.647215,0.450314,0.563870,1.403248,1.007620,7.788701
1,2,0,1,0,0,0,0,0,0,1,...,-0.135754,-0.112264,0.747696,-0.749060,-0.231931,-0.702751,-0.546698,0.683287,-0.839662,7.302227
2,5,0,1,0,0,1,0,0,0,1,...,-1.016579,-1.040409,-1.236757,-1.615922,-0.941361,-0.467237,-0.508036,-1.696411,1.208015,8.072495
3,10,1,1,0,1,0,0,0,0,1,...,-0.125015,-0.429909,-0.867203,-1.013009,-0.200335,-0.761220,-0.796694,0.657995,0.591953,7.038652
4,11,0,1,0,1,0,0,0,0,1,...,-1.816004,-1.698643,-1.486329,-2.073037,-1.737857,-1.508394,-1.529282,-1.303842,-0.155206,7.994244


In [39]:
X_train, X_val, y_train,y_val = train_test_split(train.iloc[:,1:-1], train.iloc[:,-1], test_size = 0.1, random_state=0)

In [26]:
kfolds = 1

In [42]:
if kfolds > 1:
    kf = KFold(train.shape[0], n_folds=kfolds)
    for i, (train_index, test_index) in enumerate(kf):
        dtest = xgb.DMatrix(test[test.columns[1:]])
        print('Fold {0}'.format(i + 1))
        X_train, X_val = train.iloc[train_index], train.iloc[test_index]
        dtrain = xgb.DMatrix(X_train[X_train.columns[1:-1]],label=X_train.loss)
        dvalid = xgb.DMatrix(X_val[X_val.columns[1:-1]],label=X_val.loss)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

        gbdt = xgb.train(xgb_params, dtrain, best_nrounds, watchlist,
                            obj=logregobj,
                            feval=xg_eval_mae,
                            early_stopping_rounds=25)
        del dtrain
        del dvalid
        gc.collect()
        allpredictions['p'+str(i)] = gbdt.predict(dtest)
        del dtest
        del gbdt
        gc.collect()
else:
    #dtest = xgb.DMatrix(test[test.columns[1:]].values)
    dvalid = xgb.DMatrix(X_val,label = y_val)
    dtrain = xgb.DMatrix(X_train, label = y_train)
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbdt = xgb.train(xgb_params, dtrain, best_nrounds, watchlist,
                         obj=logregobj,
                         feval=xg_eval_mae,
                         early_stopping_rounds=25)
    print mean_absolute_error(((np.exp(y_val))-200),(np.exp(gbdt.predict(dvalid))-200))
    #allpredictions['p1'] = gbdt.predict(dtest)
    del dtrain
    del dtest
    del gbdt
    gc.collect()

#print(allpredictions.head())

Fold 1


Will train until eval error hasn't decreased in 25 rounds.
[0]	train-mae:3188.223145	eval-mae:3182.067871
[1]	train-mae:3076.520508	eval-mae:3070.118896
[2]	train-mae:2926.947998	eval-mae:2920.026855
[3]	train-mae:2760.968994	eval-mae:2753.555420
[4]	train-mae:2589.693848	eval-mae:2581.534668
[5]	train-mae:2424.083008	eval-mae:2415.276611
[6]	train-mae:2268.182861	eval-mae:2258.820557
[7]	train-mae:2124.280762	eval-mae:2114.700684
[8]	train-mae:1994.265869	eval-mae:1985.179443
[9]	train-mae:1877.138184	eval-mae:1868.383179
[10]	train-mae:1773.399658	eval-mae:1765.806763
[11]	train-mae:1682.202759	eval-mae:1676.558350
[12]	train-mae:1603.258545	eval-mae:1599.835449
[13]	train-mae:1534.758179	eval-mae:1534.042969
[14]	train-mae:1476.227295	eval-mae:1478.187744
[15]	train-mae:1425.649902	eval-mae:1429.964600
[16]	train-mae:1382.530640	eval-mae:1388.938232
[17]	train-mae:1345.548218	eval-mae:1354.162476
[18]	train-mae:1314.215698	eval-mae:1325.057373
[19]	train-mae:1287.316772	eval-mae:130

Fold 2


Will train until eval error hasn't decreased in 25 rounds.
[0]	train-mae:3186.884277	eval-mae:3187.834229
[1]	train-mae:3075.301025	eval-mae:3076.104980
[2]	train-mae:2926.093750	eval-mae:2926.948975
[3]	train-mae:2759.505127	eval-mae:2760.211182
[4]	train-mae:2589.723877	eval-mae:2590.446533
[5]	train-mae:2424.015137	eval-mae:2424.974121
[6]	train-mae:2267.521240	eval-mae:2269.137207
[7]	train-mae:2122.578857	eval-mae:2124.713135
[8]	train-mae:1993.062134	eval-mae:1995.890015
[9]	train-mae:1876.710327	eval-mae:1880.375122
[10]	train-mae:1772.835693	eval-mae:1777.369019
[11]	train-mae:1682.082397	eval-mae:1687.388184
[12]	train-mae:1602.898315	eval-mae:1609.420898
[13]	train-mae:1534.809082	eval-mae:1542.701416
[14]	train-mae:1476.009766	eval-mae:1485.245483
[15]	train-mae:1425.547363	eval-mae:1436.509766
[16]	train-mae:1382.081909	eval-mae:1394.480347
[17]	train-mae:1345.945557	eval-mae:1359.574097
[18]	train-mae:1314.784180	eval-mae:1329.850220
[19]	train-mae:1288.410522	eval-mae:130

Fold 3


Will train until eval error hasn't decreased in 25 rounds.
[0]	train-mae:3184.548340	eval-mae:3197.471436
[1]	train-mae:3073.072021	eval-mae:3086.125732
[2]	train-mae:2923.979248	eval-mae:2937.350342
[3]	train-mae:2757.820312	eval-mae:2771.503418
[4]	train-mae:2587.673584	eval-mae:2601.632568
[5]	train-mae:2422.003174	eval-mae:2436.452881
[6]	train-mae:2266.151611	eval-mae:2281.280518
[7]	train-mae:2122.335449	eval-mae:2138.089111
[8]	train-mae:1992.214844	eval-mae:2008.838989
[9]	train-mae:1875.395386	eval-mae:1893.440308
[10]	train-mae:1771.131836	eval-mae:1790.849854
[11]	train-mae:1680.104614	eval-mae:1701.247192
[12]	train-mae:1601.750366	eval-mae:1624.218384
[13]	train-mae:1533.356934	eval-mae:1557.124390
[14]	train-mae:1474.954956	eval-mae:1500.467529
[15]	train-mae:1424.557129	eval-mae:1451.949219
[16]	train-mae:1381.198608	eval-mae:1410.120239
[17]	train-mae:1344.684814	eval-mae:1375.332886
[18]	train-mae:1313.304443	eval-mae:1345.258301
[19]	train-mae:1286.081421	eval-mae:131

Fold 4


Will train until eval error hasn't decreased in 25 rounds.
[0]	train-mae:3185.954346	eval-mae:3191.943115
[1]	train-mae:3075.730713	eval-mae:3081.895752
[2]	train-mae:2926.257324	eval-mae:2932.371582
[3]	train-mae:2759.001221	eval-mae:2764.948975
[4]	train-mae:2588.197754	eval-mae:2594.354492
[5]	train-mae:2421.023438	eval-mae:2427.363037
[6]	train-mae:2264.074463	eval-mae:2270.848633
[7]	train-mae:2119.577881	eval-mae:2127.162598
[8]	train-mae:1988.101562	eval-mae:1996.929810
[9]	train-mae:1871.664307	eval-mae:1882.122559
[10]	train-mae:1767.872437	eval-mae:1780.374023
[11]	train-mae:1677.421997	eval-mae:1691.853882
[12]	train-mae:1599.105713	eval-mae:1615.754639
[13]	train-mae:1530.412720	eval-mae:1549.158325
[14]	train-mae:1472.254761	eval-mae:1492.823242
[15]	train-mae:1421.597290	eval-mae:1444.057861
[16]	train-mae:1379.253784	eval-mae:1403.041626
[17]	train-mae:1343.738770	eval-mae:1368.971191
[18]	train-mae:1312.616211	eval-mae:1339.604736
[19]	train-mae:1286.115479	eval-mae:131

Fold 5


Will train until eval error hasn't decreased in 25 rounds.
[0]	train-mae:3189.914062	eval-mae:3176.199707
[1]	train-mae:3079.788086	eval-mae:3066.241699
[2]	train-mae:2930.453125	eval-mae:2917.295654
[3]	train-mae:2763.749268	eval-mae:2750.817139
[4]	train-mae:2593.195312	eval-mae:2581.004150
[5]	train-mae:2426.493896	eval-mae:2415.244629
[6]	train-mae:2269.465820	eval-mae:2259.145996
[7]	train-mae:2125.014648	eval-mae:2115.965088
[8]	train-mae:1994.257324	eval-mae:1986.709473
[9]	train-mae:1877.613403	eval-mae:1871.481323
[10]	train-mae:1773.151001	eval-mae:1768.141724
[11]	train-mae:1682.588379	eval-mae:1678.842529
[12]	train-mae:1604.149292	eval-mae:1601.509521
[13]	train-mae:1536.007812	eval-mae:1534.552368
[14]	train-mae:1477.842773	eval-mae:1477.649414
[15]	train-mae:1427.377197	eval-mae:1428.462891
[16]	train-mae:1384.948853	eval-mae:1387.201172
[17]	train-mae:1348.261963	eval-mae:1352.355469
[18]	train-mae:1316.704224	eval-mae:1322.673706
[19]	train-mae:1289.968018	eval-mae:129

In [46]:
kfolds = 1

In [47]:
if kfolds > 1:
    kf = KFold(train.shape[0], n_folds=kfolds)
    for i, (train_index, test_index) in enumerate(kf):
        dtest = xgb.DMatrix(test[test.columns[1:]])
        print('Fold {0}'.format(i + 1))
        X_train, X_val = train.iloc[train_index], train.iloc[test_index]
        dtrain = xgb.DMatrix(X_train[X_train.columns[1:-1]],label=X_train.loss)
        dvalid = xgb.DMatrix(X_val[X_val.columns[1:-1]],label=X_val.loss)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

        gbdt = xgb.train(xgb_params, dtrain, best_nrounds, watchlist,
                            obj=logregobj,
                            feval=xg_eval_mae,
                            early_stopping_rounds=25)
        del dtrain
        del dvalid
        gc.collect()
        allpredictions['p'+str(i)] = gbdt.predict(dtest)
        del dtest
        del gbdt
        gc.collect()
else:
    #dtest = xgb.DMatrix(test[test.columns[1:]].values)
    dvalid = xgb.DMatrix(X_val,label = y_val)
    dtrain = xgb.DMatrix(X_train, label = y_train)
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbdt = xgb.train(xgb_params, dtrain, best_nrounds, watchlist,
                         obj=logregobj,
                         feval=xg_eval_mae,
                         early_stopping_rounds=25)
    print mean_absolute_error(((np.exp(y_val))-200),(np.exp(gbdt.predict(dvalid))-200))
    #allpredictions['p1'] = gbdt.predict(dtest)
    del dtrain
    del dtest
    del gbdt
    gc.collect()

#print(allpredictions.head())

Will train until eval error hasn't decreased in 25 rounds.


ValueError: operands could not be broadcast together with shapes (150655,) (169486,) 

In [33]:
gbdt.predict(dtest)

array([ 7.52739954,  7.81086779,  9.23377323, ...,  7.95283222,
        7.24534369,  8.06972313], dtype=float32)

In [35]:
len(gbdt.predict(dtest))

125546

In [45]:
kfolds

5